In [19]:
%load_ext autoreload
%autoreload 2
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from utilities import data_handler
import numpy as np
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer # for tokenization only
from sklearn.decomposition import TruncatedSVD

# THE CELL BELLOW CONTAINS THE FILENAMES TO CHANGE

In [10]:
UNLABLED_DATA = '/Users/ianmagnusson/IITUDND/data/retrieved_data/harvey_extras.json'
LABLED_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/json/hurricane_harvey_final_data.json'
CLASS_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/annotations/hurricane_harvey_final_data.tsv'
NPY_OUTPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/glove/harvey/'

## Set up glove

In [4]:
conversion_file = '../models/gensim_glove.txt'
# convert glove format to work with gensim. tutorial here https://radimrehurek.com/gensim/scripts/glove2word2vec.html
# _ = glove2word2vec('/Users/ianmagnusson/IITUDND/models/glove.twitter.27B.200d.txt', conversion_file)

In [5]:
# load model, NOTE this is very slow!
glove = KeyedVectors.load_word2vec_format(conversion_file)

# Extract glove embeddings

In [21]:
data = data_handler.DataHandler(UNLABLED_DATA,LABLED_DATA,CLASS_DATA)

# 5th param is skipped because it is only needed for fitting to a full corpus
train_labeled, train_histories, test_labeled, test_histories, _, train_class, test_class  = data.get_train_test_split() 

In [22]:
EMBED_DIM = 200

def embed_tweets(tweet_jsons):
    X_embedded = np.zeros((len(tweet_jsons),EMBED_DIM))
    tokenizer = CountVectorizer().build_tokenizer()
    for i, tweet_json in enumerate(tweet_jsons):
        text = tweet_json['text'].lower()
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                X_embedded[i] += glove[token]
                num_in_vocab += 1
        X_embedded[i] = X_embedded[i] / num_in_vocab
    return X_embedded

def embed_histories(histories):
    X_embedded = np.zeros((len(histories),EMBED_DIM))
    tokenizer = CountVectorizer().build_tokenizer()
    for i, history in enumerate(histories):
        text = ' '.join([tweet_json['text'].lower() for tweet_json in history])
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                X_embedded[i] += glove[token]
                num_in_vocab += 1
        X_embedded[i] = X_embedded[i] / num_in_vocab
    return X_embedded

In [23]:
# extract!
train_len = len(train_labeled)
test_len = len(test_labeled)

X_labeled_train = embed_tweets(train_labeled)
X_histories_train = embed_histories(train_histories)
X_labeled_test = embed_tweets(test_labeled)
X_histories_test = embed_histories(test_histories)
    

In [24]:
# build class labels

y_train = np.array(train_class)
y_test = np.array(test_class)

In [25]:
# checkpoint!
np.save(NPY_OUTPUT_DIR + 'X_labeled_train.npy', X_labeled_train)
np.save(NPY_OUTPUT_DIR + 'X_histories_train.npy', X_histories_train)
np.save(NPY_OUTPUT_DIR + 'X_labeled_test.npy', X_labeled_test)
np.save(NPY_OUTPUT_DIR + 'X_histories_test.npy', X_histories_test)

np.save(NPY_OUTPUT_DIR + 'y_train.npy', y_train)
np.save(NPY_OUTPUT_DIR + 'y_test.npy', y_test)



#X_labeled_train = np.load(NPY_OUTPUT_DIR + 'X_labeled_train.npy')
#X_histories_train = np.load(NPY_OUTPUT_DIR + 'X_histories_train.npy')
#X_labeled_test = np.load(NPY_OUTPUT_DIR + 'X_labeled_test.npy')
#X_histories_test = np.load(NPY_OUTPUT_DIR + 'X_histories_test.npy')

#y_train = np.load(NPY_OUTPUT_DIR + 'y_train.npy')
#y_test = np.load(NPY_OUTPUT_DIR + 'y_test.npy')

# Reduce dimensionality with SVD (TBD)

In [14]:
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
X_SVD =svd.fit_transform(X_labeled_train)

/Users/ianmagnusson/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/truncated_svd.py:192: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/ianmagnusson/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/truncated_svd.py:192: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [15]:
svd.singular_values_


array([5.29150262e+00, 8.23780443e-33])